### Symbols in Market Watch

In [7]:
import MetaTrader5 as mt5
from dotenv import load_dotenv
import os
import csv

# -----------------------------
# Setup & MT5 initialization
# -----------------------------
load_dotenv()
login = int(os.getenv("AQUA_MT5_LOGIN"))
password = os.getenv("AQUA_MT5_PASSWORD")
server = os.getenv("AQUA_MT5_SERVER")

if not mt5.initialize(login=login, password=password, server=server):
    print("initialize() failed, error code =", mt5.last_error())
    quit()

# Fetch all symbols currently visible in Market Watch
symbols = mt5.symbols_get()

# Filter only active tradable symbols
active_symbols = [
    sym.name
    for sym in symbols
    if sym.visible and sym.trade_mode == mt5.SYMBOL_TRADE_MODE_FULL
]

# Print active symbols
print("Active symbols in Market Watch:")
for name in active_symbols:
    print(name)

# -----------------------------
# Export to CSV
# -----------------------------
csv_filename = "active_symbols.csv"

with open(csv_filename, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Symbol"])  # header
    for name in active_symbols:
        writer.writerow([name])

print(f"\nExported {len(active_symbols)} symbols to {csv_filename}")

# Shutdown MT5 connection
mt5.shutdown()

Active symbols in Market Watch:
EURUSD
GBPUSD
USDCHF
USDJPY
USDCAD
AUDUSD
NZDUSD
XAGUSD
XAUUSD
BRENT
WTI
AUS200
GER40
JPN225
UK100
US30
SPX500
NAS100

Exported 18 symbols to active_symbols.csv


True

### Max Sharpe Weights

In [13]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from dotenv import load_dotenv
import os

# -----------------------------
# Setup & MT5 initialization
# -----------------------------
load_dotenv()
login = int(os.getenv("AQUA_MT5_LOGIN"))
password = os.getenv("AQUA_MT5_PASSWORD")
server = os.getenv("AQUA_MT5_SERVER")

if not mt5.initialize(login=login, password=password, server=server):
    print("initialize() failed, error code =", mt5.last_error())
    quit()

# -----------------------------
# Config
# -----------------------------
risk_free_rate = 0.0  # annual risk-free rate

# FX pairs needed to convert index returns to USD
fx_map = {
    "GER40": "EURUSD",
    "JPN225": "USDJPY",   # flip sign
    "UK100": "GBPUSD",
    "AUS200": "AUDUSD"
}

# -----------------------------
# Helpers
# -----------------------------
def fetch_mt5_data(symbol, days=60):
    utc_to = pd.Timestamp.now()
    utc_from = utc_to - pd.Timedelta(days=days)

    rates = mt5.copy_rates_range(symbol, mt5.TIMEFRAME_D1, utc_from, utc_to)
    if rates is None:
        raise ValueError(f"Failed to fetch data for {symbol}")

    df = pd.DataFrame(rates)
    df["time"] = pd.to_datetime(df["time"], unit="s")
    df.set_index("time", inplace=True)
    return df

def calculate_log_returns(prices: pd.Series) -> pd.Series:
    return np.log(prices / prices.shift(1)).dropna()

# -----------------------------
# Load expected returns
# -----------------------------
# ai_factor_signal_aqua.csv:  asset \t factor_signal
expected_returns_df = pd.read_csv("ai_factor_signal_aqua.csv", sep="\t", header=None)
expected_returns_df.columns = ["asset", "factor_signal"]

# -----------------------------
# Z-score the factor signals
# -----------------------------
mean_signal = expected_returns_df["factor_signal"].mean()
std_signal = expected_returns_df["factor_signal"].std()

expected_returns_df["zscore_signal"] = (expected_returns_df["factor_signal"] - mean_signal) / std_signal

# Convert to dict
expected_returns_dict = dict(zip(expected_returns_df["asset"], expected_returns_df["zscore_signal"]))

symbols = list(expected_returns_dict.keys())

# -----------------------------
# Fetch price data for assets
# -----------------------------
data = {symbol: fetch_mt5_data(symbol) for symbol in symbols}

# -----------------------------
# Fetch FX data and compute FX log returns
# -----------------------------
fx_symbols = set(fx_map[s] for s in symbols if s in fx_map)
fx_data = {fx: fetch_mt5_data(fx) for fx in fx_symbols}

fx_returns = {}
for fx, df_fx in fx_data.items():
    lr = calculate_log_returns(df_fx["close"])

    # If pair is quoted as USDXXX (e.g., USDJPY, USDHKD), flip sign to get XXXUSD log return
    if fx.startswith("USD"):
        lr = -lr  # log(1/x) = -log(x)

    fx_returns[fx] = lr

# -----------------------------
# Compute USD-adjusted log returns for all assets
# -----------------------------
asset_returns = {}

for symbol in symbols:
    idx_lr = calculate_log_returns(data[symbol]["close"])

    if symbol in fx_map:
        fx_symbol = fx_map[symbol]
        fx_lr = fx_returns[fx_symbol]

        # Align index and FX by date
        combined = pd.concat([idx_lr, fx_lr], axis=1, join="inner")
        combined.columns = ["idx", "fx"]

        # USD-adjusted return: r_usd = r_index_local + r_fx
        asset_returns[symbol] = combined["idx"] + combined["fx"]
    else:
        # Already in USD terms
        asset_returns[symbol] = idx_lr

# -----------------------------
# Build aligned returns matrix
# -----------------------------
returns = pd.DataFrame(asset_returns).dropna(how="any")

# -----------------------------
# Covariance matrix (daily, log returns)
# -----------------------------
cov_matrix = returns.cov()

# Expected daily returns vector (assumed to be in USD terms already)
expected_returns = np.array([expected_returns_dict[symbol] for symbol in symbols])
expected_returns = expected_returns * returns.std().values  # scale by asset volatilities

# -----------------------------
# Max Sharpe optimization
# -----------------------------
def max_sharpe_ratio(expected_returns, cov_matrix, risk_free_rate=0.0):
    num_assets = len(expected_returns)

    def neg_sharpe(weights):
        port_ret = np.dot(weights, expected_returns)
        port_vol = np.sqrt(weights.T @ cov_matrix @ weights)
        return -(port_ret - risk_free_rate) / port_vol

    # Dollar-neutral: sum(weights) = 0
    constraints = [{"type": "eq", "fun": lambda w: np.sum(w)}]

    # Allow long/short
    bounds = [(-1, 1)] * num_assets

    # Start with zero‑sum initial guess
    init_guess = np.ones(num_assets)
    init_guess[: num_assets // 2] = 1
    init_guess[num_assets // 2 :] = -1
    init_guess = init_guess / np.sum(np.abs(init_guess))

    result = minimize(neg_sharpe, init_guess, bounds=bounds, constraints=constraints)

    if not result.success:
        raise ValueError(f"Optimization failed: {result.message}")

    weights = result.x
    daily_ret = np.dot(weights, expected_returns)
    daily_vol = np.sqrt(weights.T @ cov_matrix @ weights)

    return weights, daily_ret, daily_vol

# -----------------------------
# Run optimization
# -----------------------------
weights, daily_port_return, daily_port_vol = max_sharpe_ratio(
    expected_returns, cov_matrix, risk_free_rate=risk_free_rate
)

# -----------------------------
# Export optimal weights
# -----------------------------
weights_df = pd.DataFrame({
    "Symbol": symbols,
    "Weight": weights
})
weights_df.to_csv("aqua_funded_optimal_portfolio_weights.csv", header=False, index=False)
print("\nWeights exported to aqua_funded_optimal_portfolio_weights.csv")

# -----------------------------
# Annualized statistics (using daily log returns)
# -----------------------------
annual_port_return = (1 + daily_port_return)**252 - 1
annual_port_vol = daily_port_vol * np.sqrt(252)
annual_sharpe = (annual_port_return - risk_free_rate) / annual_port_vol

# -----------------------------
# Output
# -----------------------------
print("\nOptimal Portfolio Weights:")
for symbol, weight in zip(symbols, weights):
    print(f"{symbol}: {weight:.4f}")

print("\n--- Portfolio Statistics ---")
print(f"Daily Portfolio Return: {daily_port_return:.4%}")
print(f"Annualized Portfolio Return: {annual_port_return:.4%}")
print(f"Daily Portfolio Volatility: {daily_port_vol:.4%}")
print(f"Annualized Portfolio Volatility: {annual_port_vol:.4%}")
print(f"Annualized Sharpe Ratio: {annual_sharpe:.2f}")

# -----------------------------
# Shutdown MT5
# -----------------------------
mt5.shutdown()


Weights exported to aqua_funded_optimal_portfolio_weights.csv

Optimal Portfolio Weights:
AUDUSD: 0.0784
EURUSD: -0.1774
GBPUSD: 0.0535
NZDUSD: 0.1121
USDJPY: -0.0394
USDCHF: -0.2150
USDCAD: 0.1744
XAGUSD: 0.0178
XAUUSD: -0.0064
WTI: -0.1185
BRENT: 0.0877
SPX500: 1.0000
US30: -0.1213
NAS100: -0.4188
GER40: -0.0290
JPN225: -0.0072
UK100: -0.1109
AUS200: -0.2799

--- Portfolio Statistics ---
Daily Portfolio Return: 0.5169%
Annualized Portfolio Return: 266.6598%
Daily Portfolio Volatility: 0.1248%
Annualized Portfolio Volatility: 1.9806%
Annualized Sharpe Ratio: 134.64


True

### Volatility Targeted Weights using EWMA estimates

In [9]:
import pandas as pd
import numpy as np

# --- Load weights from comma-separated file ---
weights_df = pd.read_csv(
    "aqua_funded_optimal_portfolio_weights.csv",
    sep=",",
    header=None,
    engine="python",
    encoding="utf-8-sig"
)

weights_df.columns = ["asset", "weight"]

# Align weights to returns.columns order
weights_df = weights_df.set_index("asset").reindex(returns.columns)

# Extract weight vector
weights = weights_df["weight"].values

print("Loaded weights (aligned):")
print(weights)
print("\nAsset order:")
print(list(returns.columns))

# ============================================================
# --- EWMA COVARIANCE MATRIX ---
# ============================================================

lambda_ = 0.94  # RiskMetrics default
rets = returns.values
n = rets.shape[1]

# Start with sample covariance as initial value
ewma_cov = returns.cov().values.copy()

# Iterate through returns to update EWMA covariance
for t in range(1, len(rets)):
    r = rets[t-1].reshape(-1, 1)
    ewma_cov = lambda_ * ewma_cov + (1 - lambda_) * (r @ r.T)

# Portfolio EWMA volatility
ewma_daily_vol = np.sqrt(weights.T @ ewma_cov @ weights)
ewma_annual_vol = ewma_daily_vol * np.sqrt(252)
daily_returns = returns @ weights
annual_return = daily_returns.mean() * 252

print("\n--- EWMA VOLATILITY FORECAST ---")
print(f"EWMA Daily Volatility: {ewma_daily_vol:0.2%}")
print(f"EWMA Annual Volatility: {ewma_annual_vol:0.2%}")
print(f"EWMA Daily Portfolio Return: {daily_returns.mean():0.2%}")
print(f"EWMA Annual Portfolio Return: {annual_return:0.2%}")

# ============================================================
# --- SCALE PORTFOLIO TO TARGET VOLATILITY USING EWMA ---
# ============================================================

target_vol = 0.10  # 10% annual volatility

scale_factor = target_vol / ewma_annual_vol
scaled_weights = weights * scale_factor

# Recompute volatility after scaling
scaled_daily_vol = np.sqrt(scaled_weights.T @ ewma_cov @ scaled_weights)
scaled_annual_vol = scaled_daily_vol * np.sqrt(252)

# Compute scaled returns
scaled_daily_returns = returns @ scaled_weights
scaled_avg_daily_return = scaled_daily_returns.mean()
scaled_annual_return = scaled_avg_daily_return * 252

print(f"\n--- SCALED PORTFOLIO (Target Vol = {target_vol:.0%}) ---")
print(f"Scale Factor: {scale_factor:.4f}")
print(f"Scaled Daily Volatility (EWMA): {scaled_daily_vol:0.2%}")
print(f"Scaled Annual Volatility (EWMA): {scaled_annual_vol:0.2%}")
print(f"Scaled Daily Portfolio Return: {scaled_avg_daily_return:0.2%}")
print(f"Scaled Annual Portfolio Return: {scaled_annual_return:0.2%}")

print("\nScaled Weights:")
for asset, w in zip(returns.columns, scaled_weights):
    print(f"{asset}: {w:.6f}")

# --- EXPORT SCALED WEIGHTS TO CSV ---
scaled_df = pd.DataFrame({
    "asset": returns.columns,
    "scaled_weight": scaled_weights
})

scaled_df.to_csv("aqua_funded_scaled_weights.csv", index=False)
print("\nScaled weights exported to aqua_funded_scaled_weights.csv")

Loaded weights (aligned):
[-0.54292738 -0.12224188  0.03914165  0.09240455  0.05726182 -0.42643512
  0.52886674  0.02700176  0.05656028 -0.38156933  0.27634253  1.
 -0.10263983 -0.15068135 -0.27256705 -0.03380239  0.21374412 -0.25845911]

Asset order:
['AUDUSD', 'EURUSD', 'GBPUSD', 'NZDUSD', 'USDJPY', 'USDCHF', 'USDCAD', 'XAGUSD', 'XAUUSD', 'WTI', 'BRENT', 'SPX500', 'US30', 'NAS100', 'GER40', 'JPN225', 'UK100', 'AUS200']

--- EWMA VOLATILITY FORECAST ---
EWMA Daily Volatility: 0.30%
EWMA Annual Volatility: 4.81%
EWMA Daily Portfolio Return: 0.15%
EWMA Annual Portfolio Return: 38.15%

--- SCALED PORTFOLIO (Target Vol = 10%) ---
Scale Factor: 2.0796
Scaled Daily Volatility (EWMA): 0.63%
Scaled Annual Volatility (EWMA): 10.00%
Scaled Daily Portfolio Return: 0.31%
Scaled Annual Portfolio Return: 79.33%

Scaled Weights:
AUDUSD: -1.129074
EURUSD: -0.254215
GBPUSD: 0.081399
NZDUSD: 0.192165
USDJPY: 0.119082
USDCHF: -0.886816
USDCAD: 1.099833
XAGUSD: 0.056153
XAUUSD: 0.117623
WTI: -0.793513
BR

### Lot Sizing

In [10]:
import math
import pandas as pd
import MetaTrader5 as mt5
import numpy as np

# ============================
# USER CONFIG
# ============================
EQUITY = 200000.90  # set your account equity

# FX-exempt symbols (no price in formula)
FX_EXEMPT = ["USDJPY", "USDCHF", "USDCAD"]

# Global index → FX mapping
INDEX_FX_MAP = {
    "GER40": "EURUSD",
    "JPN225": "USDJPY",
    "UK100": "GBPUSD",
    "AUS200": "AUDUSD"
}

# ============================
# MT5 INITIALIZATION
# ============================
mt5.initialize()

def get_latest_price(symbol):
    tick = mt5.symbol_info_tick(symbol)
    if tick is None:
        return None
    return tick.ask

def fetch_prices(assets):
    return {a: get_latest_price(a) for a in assets}

def fetch_index_fx_rates():
    return {idx: get_latest_price(fx) for idx, fx in INDEX_FX_MAP.items()}

# ============================
# LOAD CSV FILES
# ============================
contract_df = pd.read_csv("aqua_funded_contract_size.csv")
weights_df = pd.read_csv("aqua_funded_scaled_weights.csv")

# Normalize column names
contract_df.columns = contract_df.columns.str.strip().str.lower()
weights_df.columns = weights_df.columns.str.strip().str.lower()

# Rename scaled_weight → weight
weights_df.rename(columns={"scaled_weight": "weight"}, inplace=True)

# Merge contract sizes + weights
df = contract_df.merge(weights_df, on="asset", how="left")

# ============================
# FETCH PRICES FROM MT5
# ============================
all_assets = df["asset"].tolist()
latest_prices = fetch_prices(all_assets)
index_fx_rates = fetch_index_fx_rates()

df["latest_price"] = df["asset"].map(latest_prices)

# ============================
# LOT SIZE CALCULATION
# ============================
def compute_lot(row):
    asset = row["asset"]
    weight = row["weight"]
    contract_size = row["contract_size"]
    price = row["latest_price"]

    # Missing weight or contract size
    if pd.isna(weight) or pd.isna(contract_size):
        return None

    # Contract size cannot be zero
    if contract_size == 0:
        return None

    # Rule 1: FX-exempt assets
    if asset in FX_EXEMPT:
        return (weight * EQUITY) / contract_size

    # Missing or zero price
    if price is None or price == 0:
        return None

    # Rule 2: Global index → convert to USD
    if asset in INDEX_FX_MAP:
        fx_rate = index_fx_rates.get(asset)
        if fx_rate is None or fx_rate == 0:
            return None
        # Invert FX rate for JP225
        if asset in ["JPN225"]:
            fx_rate = 1 / fx_rate

        price = price * fx_rate

        # After conversion, price still cannot be zero
        if price == 0:
            return None

    # Final safety check
    denominator = price * contract_size
    if denominator == 0:
        return None

    return (weight * EQUITY) / denominator

from decimal import Decimal, ROUND_UP

df["lot_size"] = df.apply(compute_lot, axis=1)

df["lot_size"] = df["lot_size"].apply(
    lambda x: Decimal(str(x)).quantize(Decimal("0.01"), rounding=ROUND_UP)
)




# ============================
# OUTPUT
# ============================
print(df[["asset", "weight", "contract_size", "latest_price", "lot_size"]].dropna())
df[["asset", "weight", "contract_size", "latest_price", "lot_size"]]\
    .dropna()\
    .to_csv("aqua_funded_lot_sizes_output.csv", index=False)

# ============================
# GROSS TOTAL LOT SIZE
# ============================
df["abs_lot_size"] = df["lot_size"].abs()
gross_total_lot_size = df["abs_lot_size"].sum()

print(f"\nGross Total Lot Size (absolute): {gross_total_lot_size:.4f}")
mt5.shutdown()

     asset    weight  contract_size  latest_price lot_size
0   AUDUSD -1.129074         100000       0.66913    -3.38
1   EURUSD -0.254215         100000       1.16397    -0.44
2   GBPUSD  0.081399         100000       1.34095     0.13
3   NZDUSD  0.192165         100000       0.57359     0.68
4   USDJPY  0.119082         100000     157.94900     0.24
5   USDCHF -0.886816         100000       0.80157    -1.78
6   USDCAD  1.099833         100000       1.39197     2.20
7   XAGUSD  0.056153           5000      79.97100     0.03
8   XAUUSD  0.117623            100    4509.98000     0.06
9      WTI -0.793513            100      58.81000   -26.99
10   BRENT  0.574683            100      63.29000    18.17
11  SPX500  2.079603             10    6965.80000     5.98
12    US30 -0.213450              1   49512.00000    -0.87
13  NAS100 -0.313357             10   25761.24000    -0.25
14   GER40 -0.566831              1   25280.90000    -3.86
15  JPN225 -0.070296            100   53535.00000    -0.

True

In [12]:
returns

,AUDUSD,EURUSD,GBPUSD,NZDUSD,USDJPY,USDCHF,USDCAD,XAGUSD,XAUUSD,WTI,BRENT,SPX500,US30,NAS100,GER40,JPN225,UK100,AUS200
time,,,,,,,,,,,,,,,,,,
2025-09-09,-0.001184,-0.004499,-0.001226,-0.002411,-0.000814,0.005069,0.002995,-0.010977,-0.002900,0.004614,0.004500,0.003625,0.037988,0.004011,-0.004247,-0.007616,0.001900,-0.004473
2025-09-10,0.004426,-0.001325,0.000222,0.002343,0.000353,0.002481,0.001213,0.006586,0.003941,0.014810,0.015004,0.001210,-0.003818,-0.001162,-0.010099,0.005777,-0.001887,0.005334
2025-09-11,0.007127,0.003526,0.003048,0.006295,-0.001581,-0.004314,-0.002087,0.009582,-0.001826,-0.024059,-0.019505,0.008341,0.013560,0.006238,0.010310,0.022118,0.011369,0.011769
2025-09-12,-0.002014,-0.000102,-0.001504,-0.003738,0.002815,0.000151,0.000809,0.014915,0.002389,0.004954,0.008832,-0.000182,-0.005895,0.004781,-0.001914,-0.004917,-0.007144,-0.006429
2025-09-15,0.003439,0.002256,0.003433,0.002566,-0.001579,-0.001711,-0.004721,0.011910,0.009851,0.009519,0.007572,0.004274,0.000458,0.007728,0.003074,0.007049,0.004318,0.011913
2025-09-16,0.002202,0.008922,0.003377,0.002861,-0.006480,-0.011114,-0.002871,-0.002794,0.002994,0.019391,0.015120,-0.000620,-0.001484,-0.000045,-0.005155,0.003246,-0.004277,-0.003779
2025-09-17,-0.004844,-0.004553,-0.001474,-0.004101,0.003558,0.003964,0.002544,-0.021050,-0.008137,-0.009023,-0.008927,-0.000469,0.005358,-0.001055,-0.004608,-0.001305,-0.001040,-0.009836
2025-09-18,-0.006062,-0.002238,-0.005233,-0.013493,0.006916,0.004466,0.001625,0.003332,-0.004239,-0.005013,-0.006193,0.005104,0.003513,0.008883,0.011209,0.014513,-0.001232,-0.005607
2025-09-19,-0.003485,-0.003715,-0.006336,-0.004498,-0.000257,0.003931,-0.001560,0.029753,0.011122,-0.015352,-0.013101,0.003354,0.002249,0.005925,-0.006198,-0.011989,-0.008513,-0.001100
